In [2]:
import os
import pandas as pd
import numpy as np
import json
from io import StringIO

In [3]:
root_dir = os.path.dirname(os.getcwd())
df = pd.read_json("{}/experiments/E1/data/data_E1_raw.json".format(root_dir))

data = pd.DataFrame()
for idx,row in enumerate(df.iterrows()):
    row_data = StringIO(row[1].content)
    dataframe = pd.read_csv(row_data, sep=',')
    dataframe.insert(0,"subject_id",np.repeat(idx+1,dataframe.shape[0]))
    data = pd.concat([data,dataframe],ignore_index=True)

#data = data[["subject_id","rt","stimulus","trial_type","sampling","status","nb_of_run","stim_type","length","state_OR_event",
                   #"branch_first","time_interval","node_selected"]]
data = data.drop(['button_pressed','trial_index', 'internal_node_id','question_order'],axis=1)
#data = data[(data["trial_type"]=="causal-graph1") & (data["status"]=="active")].drop(["trial_type","status"],axis=1)
data.loc[data['status']!='active','stimulus'] = 'instruction'
data.loc[data['trial_type']=='survey-text','stimulus'] = 'comment'
def clean_rt(stim, rt):
    if "instruction" in stim:
        return np.nan
    else:
        return rt
    
data['rt'] = data.apply(lambda x: clean_rt(x['stimulus'],x['rt']),axis=1)
#data.loc[(data['stimulus']=='instruction') | (data['stimulus']=='comment'),'rt'] = np.nan
#data['rt'] = np.where((data['stimulus']=='instruction') | (data['stimulus']=='comment'),np.nan,data['rt'])

stimulus = []
for items in data.stimulus.array:
    stim = items.split(',')
    if len(stim) == 1:
        stim = stim[0]
    else:
        stim = stim[1].split('/')[1].split('.')[0]
    stimulus.append(stim)
    
stimulus = pd.Series(stimulus)
data.stimulus = stimulus
data = data.drop(['status'],axis=1)
data.to_csv(r"{}/data/data_E1.csv".format(root_dir),index=False)